In [4]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px
import seaborn as sns
import missingno as msno
import warnings
import os
from functools import lru_cache
from math import ceil
from itertools import zip_longest
from typing import List, Tuple, Union, Any
from scipy.stats import chi2_contingency
from itertools import combinations
from collections import defaultdict
from statsmodels.stats.outliers_influence import variance_inflation_factor
import scipy.stats as stats
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.feature_selection import r_regression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso 
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet 
from scipy.stats import skew
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier  # Make sure CatBoost is imported
from lightgbm import LGBMClassifier
import statsmodels.api as sm
import lightgbm as lgb

In [5]:
data= pd.read_csv("Ames_HousePrice.csv")

In [3]:
data.shape

(2580, 82)

In [9]:
#Identify categorical data types and rank them in the order in which they are most affected by missing data

missing_data = data.isnull().sum()
missing_percentage = (missing_data / len(data)) * 100
data_type = data.dtypes
 
# Combining the counts and percentages into a DataFrame for better visualization
missing_info = pd.DataFrame({'Missing Values': missing_data, 'Percentage': missing_percentage,
                             'Data Type':data_type})
 
# Sorting the DataFrame by the percentage of missing values in descending order
missing_info = missing_info.sort_values(by='Percentage', ascending=False)
 
# Display columns with missing values of 'object' data type
print(missing_info[(missing_info['Missing Values'] > 0) & (missing_info['Data Type'] == 'object')])

              Missing Values  Percentage Data Type
PoolQC                  2571   99.651163    object
MiscFeature             2483   96.240310    object
Alley                   2412   93.488372    object
Fence                   2055   79.651163    object
MasVnrType              1573   60.968992    object
FireplaceQu             1241   48.100775    object
GarageCond               129    5.000000    object
GarageFinish             129    5.000000    object
GarageQual               129    5.000000    object
GarageType               127    4.922481    object
BsmtExposure              71    2.751938    object
BsmtFinType2              70    2.713178    object
BsmtFinType1              69    2.674419    object
BsmtCond                  69    2.674419    object
BsmtQual                  69    2.674419    object
Electrical                 1    0.038760    object


In [10]:
# Imputing Missing Categorical Data 
mode_value = data['Electrical'].mode()[0]
data['Electrical'].fillna(mode_value, inplace=True)
 
missing_categorical = missing_info[(missing_info['Missing Values'] > 0)
                           & (missing_info['Data Type'] == 'object')]
 
for item in missing_categorical.index.tolist():
    data[item].fillna("None", inplace=True)
 
print(data[missing_categorical.index].isnull().sum())

PoolQC          0
MiscFeature     0
Alley           0
Fence           0
MasVnrType      0
FireplaceQu     0
GarageCond      0
GarageFinish    0
GarageQual      0
GarageType      0
BsmtExposure    0
BsmtFinType2    0
BsmtFinType1    0
BsmtCond        0
BsmtQual        0
Electrical      0
dtype: int64


In [7]:
# Identify numerical data types and rank them in the order in which they are most affected by missing data. 
# Calculating the percentage of missing values for each column
missing_data = data.isnull().sum()
missing_percentage = (missing_data / len(data)) * 100
data_type = data.dtypes
 
# Combining the counts and percentages into a DataFrame for better visualization
missing_info = pd.DataFrame({'Missing Values': missing_data, 'Percentage': missing_percentage,
                             'Data Type':data_type})
 
# Sorting the DataFrame by the percentage of missing values in descending order
missing_info = missing_info.sort_values(by='Percentage', ascending=False)
 
# Display columns with missing values of numeric data type
print(missing_info[(missing_info['Missing Values'] > 0)
                   & (missing_info['Data Type'] == np.number)])

              Missing Values  Percentage Data Type
LotFrontage              462   17.906977   float64
GarageYrBlt              129    5.000000   float64
MasVnrArea                14    0.542636   float64
BsmtHalfBath               2    0.077519   float64
BsmtFullBath               2    0.077519   float64
GarageArea                 1    0.038760   float64
GarageCars                 1    0.038760   float64
BsmtUnfSF                  1    0.038760   float64
BsmtFinSF2                 1    0.038760   float64
BsmtFinSF1                 1    0.038760   float64
TotalBsmtSF                1    0.038760   float64


In [8]:
#Imputing  Missing numerical Values
# Initialize a DataFrame to store the concise information
concise_info = pd.DataFrame(columns=['Feature', 'Missing Values After Imputation', 
                                     'Mean Value Used to Impute'])
 
# Identify and impute missing numerical values, and store the related concise information
missing_numeric_feature = missing_info[(missing_info['Missing Values'] > 0)
                           & (missing_info['Data Type'] == np.number)]
 
for item in missing_numeric_feature.index.tolist():
    mean_value = data[item].mean(skipna=True)
    data[item].fillna(mean_value, inplace=True)
 
    # Append the concise information to the concise_info DataFrame
    concise_info.loc[len(concise_info)] = pd.Series({
        'Feature': item,
        'Missing Values After Imputation': data[item].isnull().sum(),
        'Mean Value Used to Impute': mean_value
    })
 
# Display the concise_info DataFrame
print(concise_info)

         Feature Missing Values After Imputation Mean Value Used to Impute
0    LotFrontage                               0                 68.516053
1    GarageYrBlt                               0               1976.982048
2     MasVnrArea                               0                 99.308262
3   BsmtHalfBath                               0                  0.062064
4   BsmtFullBath                               0                  0.435221
5     GarageArea                               0                466.842575
6     GarageCars                               0                  1.747577
7      BsmtUnfSF                               0                 539.10159
8     BsmtFinSF2                               0                 53.238852
9     BsmtFinSF1                               0                444.346258
10   TotalBsmtSF                               0                 1036.6867
